In [41]:
import numpy as np
from astropy.io import fits
import os
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from glob import glob
import pandas as pd
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

1 r500 NFW

In [57]:
datapath = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/dom'
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    for i in range(len(dat['ID'])):
        subdir = f"{rootdir}/SDSSTG{dat['ID'][i]}"
        if os.path.exists(subdir):
            savepath = f'{subdir}/fk5_regs'
            peras = cosmo.arcsec_per_kpc_proper(dat['z'][i])
            rad = np.round((peras*dat['R500_NFW'][i]* u.kpc ).to_value(u.arcmin), decimals=4)
            with open(f'{savepath}/R500_fk5.reg', 'w') as newf:
                newf.write('global color=magenta dashlist=8 3 width=2 font="helvetica 12 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
                newf.write('fk5 \n')
                newf.write(f'''circle({dat["RA_C"][i]}, {dat["DEC_C"][i]}, {rad}')''')
        else:
            print(subdir)


/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG9370
/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG24595


2 bkg 12.1242',14.799'

In [27]:
datapath = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/dom'
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    for i in range(len(dat['ID'])):
        subdir = f"{rootdir}/SDSSTG{dat['ID'][i]}"
        if os.path.exists(subdir):
            savepath = f'{subdir}/fk5_regs'
            os.makedirs(savepath, exist_ok=True)
            with open(f'{savepath}/bkg.reg', 'w') as newf:
                newf.write('global color=cyan dashlist=8 3 width=2 font="helvetica 12 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
                newf.write('fk5 \n')
                newf.write(f'''annulus({dat["RA_C"][i]}, {dat["DEC_C"][i]}, 12.1242',14.799')''')
        else:
            print(subdir)


/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG9370
/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG24595


3 r500 estimated from 0.7R200

In [58]:
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
datapath = f'{rootdir}/dom'

with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    doc_ids = dat['ID']
all_ids = np.array([f.split('/')[-1].split('G')[-1] for f in glob(f'{rootdir}/SDSS*')]).astype(int)
docmiss_ids = all_ids[~np.isin(all_ids, doc_ids)]

# load manually chosen group centers
cen = pd.read_csv(f'{datapath}/my_centers.csv')

# for sources not in *forecast.fits
ids = docmiss_ids
with fits.open(f'{datapath}/xgap_master_v1.1.fits') as f:
    dat = f[1].data
    for id in ids:
        subdir = f"{rootdir}/SDSSTG{id}"
        if os.path.exists(subdir):
            savepath = f'{subdir}/fk5_regs'
            os.makedirs(savepath, exist_ok=True)
            print(id)
            msk = dat['GROUP_ID']==f'{id}'
            msk_cen = cen['ID'] == f'SDSSTG{id}'
            rad = np.round((dat['R200_deg'][msk][0]* 0.7 *0.1* 60), decimals=4)
            with open(f'{savepath}/R500_01_fk5.reg', 'w') as newf:
                newf.write('global color=magenta dashlist=8 3 width=2 font="helvetica 12 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
                newf.write('fk5 \n')
                newf.write(f'''circle({np.array(cen["RA"])[msk_cen][0]}, {np.array(cen["DEC"])[msk_cen][0]}, {rad}')''')
            # also bkg
            with open(f'{savepath}/bkg.reg', 'w') as newf:
                newf.write('global color=magenta dashlist=8 3 width=2 font="helvetica 12 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
                newf.write('fk5 \n')
                newf.write(f'''annulus({np.array(cen["RA"])[msk_cen][0]}, {np.array(cen["DEC"])[msk_cen][0]}, 12.1242',14.799')''')
        else:
            print(subdir)


22635
28674
11320
3669
11631
10159
16386
16393
3513
4047


automatically open ds9, load regions and save plots

In [29]:
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    ids = dat['ID']
regionfile = f'fk5_regs/bkg.reg'
for i in range(len(ids)):
    subdir = f"{rootdir}/SDSSTG{ids[i]}"
    savepath = f'{rootdir}/figs/bkgs'
    os.makedirs(savepath, exist_ok = True)
    savefile = f'SDSSTG{ids[i]}_bkg.png'
    if os.path.exists(subdir):
        os.chdir(subdir)
        os.system(f"rm {savepath}/{savefile}")
        os.system(f'ds9 comb-fovimsky-350-2000.fits -region emllist_every.reg -region {regionfile} -scale log -cmap b -zoom to fit -smooth yes &')
        os.system(f'sleep 5')
        os.system(f"xpaset -p ds9 saveimage png {savepath}/{savefile}")
        os.system(f"xpaset -p ds9 exit")




rm: /Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/figs/bkgs/SDSSTG8050_bkg.png: No such file or directory
Fontconfig warning: ignoring UTF-8: not a valid region tag
rm: /Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/figs/bkgs/SDSSTG10842_bkg.png: No such file or directory
Fontconfig warning: ignoring UTF-8: not a valid region tag
rm: /Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/figs/bkgs/SDSSTG885_bkg.png: No such file or directory
Fontconfig warning: ignoring UTF-8: not a valid region tag
rm: /Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/figs/bkgs/SDSSTG1011_bkg.png: No such file or directory
Fontconfig warning: ignoring UTF-8: not a valid region tag
Fontconfig warning: ignoring UTF-8: not a valid region tag
XPA$ERROR no 'xpaset' access points match template: ds9
rm: /Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/figs/bkgs/SDSSTG4654_bkg.png: No such file or directory
Fontconfig warning: ignoring UTF-8: not a valid region tag
